# TSA Chapter 1: Random Walk and Trend Types

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_Ch1/TSA_ch1_random_walk/TSA_ch1_random_walk.ipynb)

This notebook demonstrates:
- Random walk properties: growing variance, multiple realizations
- White noise vs random walk comparison
- Stationary AR(1) vs random walk (ACF comparison)
- Trend types: linear, exponential, stochastic
- Unit root series: RW, RW with drift, near unit root

In [ ]:
!pip install yfinance matplotlib numpy scipy statsmodels pandas -q

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import acf

In [ ]:
# Style configuration
COLORS = {
    'blue': '#1A3A6E',
    'red': '#DC3545',
    'green': '#2E7D32',
    'orange': '#E67E22',
    'gray': '#666666',
    'purple': '#8E44AD',
}

plt.rcParams.update({
    'axes.facecolor': 'none',
    'figure.facecolor': 'none',
    'savefig.transparent': True,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.grid': False,
    'font.size': 9,
    'axes.titlesize': 10,
    'axes.labelsize': 9,
    'xtick.labelsize': 8,
    'ytick.labelsize': 8,
    'legend.fontsize': 8,
    'figure.dpi': 150,
    'lines.linewidth': 1.2,
    'axes.edgecolor': '#333333',
    'axes.linewidth': 0.8,
})

np.random.seed(42)

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    try:
        charts_path = os.path.join('..', '..', '..', 'charts', name)
        fig.savefig(f'{charts_path}.pdf', bbox_inches='tight', transparent=True, dpi=150)
        fig.savefig(f'{charts_path}.png', bbox_inches='tight', transparent=True, dpi=150)
    except Exception:
        pass
    print(f'Saved: {name}.pdf + .png')

def generate_ar1(n, phi, sigma=1.0):
    x = np.zeros(n)
    eps = np.random.normal(0, sigma, n)
    for t in range(1, n):
        x[t] = phi * x[t-1] + eps[t]
    return x

def generate_random_walk(n, sigma=1.0, drift=0.0):
    eps = np.random.normal(0, sigma, n)
    return np.cumsum(eps) + drift * np.arange(n)

def generate_white_noise(n, sigma=1.0):
    return np.random.normal(0, sigma, n)

In [ ]:
# Chart: random_walk
# Random walk realizations showing growing variance
np.random.seed(42)
n = 300

fig, axes = plt.subplots(1, 2, figsize=(8, 3.0))

colors_rw = [COLORS['blue'], COLORS['red'], COLORS['green'], COLORS['orange'], COLORS['purple']]
for i in range(5):
    rw = generate_random_walk(n)
    axes[0].plot(rw, color=colors_rw[i], linewidth=0.8, alpha=0.7,
                 label=f'Realization {i+1}')

axes[0].axhline(0, color=COLORS['gray'], linewidth=0.5, linestyle='--')
t_arr = np.arange(1, n+1)
axes[0].fill_between(t_arr-1, -2*np.sqrt(t_arr), 2*np.sqrt(t_arr),
                     alpha=0.08, color=COLORS['gray'], label=r'$\pm 2\sqrt{t}\sigma$')
axes[0].set_title('Random walk: multiple realizations', fontsize=9, fontweight='bold')
axes[0].set_xlabel('Time')
axes[0].set_ylabel(r'$X_t$')
axes[0].legend(fontsize=7, loc='upper right', frameon=False)

# Variance grows linearly
variances = []
num_sims = 500
for _ in range(num_sims):
    rw = generate_random_walk(n)
    variances.append(rw**2)
var_empirical = np.mean(variances, axis=0)
axes[1].plot(var_empirical, color=COLORS['blue'], linewidth=1.0, label=r'$\widehat{Var}(X_t)$ (empirical)')
axes[1].plot(t_arr, color=COLORS['red'], linewidth=1.0, linestyle='--', label=r'$t \cdot \sigma^2$ (theoretical)')
axes[1].set_title(r'$Var(X_t) = t\sigma^2$ grows linearly', fontsize=9, fontweight='bold')
axes[1].set_xlabel('Time (t)')
axes[1].set_ylabel('Variance')
axes[1].legend(fontsize=7, loc='upper right', frameon=False)

fig.tight_layout(w_pad=2.0)
save_chart(fig, 'random_walk')
plt.show()

In [ ]:
# Chart: ch1_wn_rw
# White noise vs random walk side by side
np.random.seed(42)
n = 300
wn = generate_white_noise(n)
rw = np.cumsum(wn)

fig, axes = plt.subplots(2, 2, figsize=(7, 4.0))

axes[0, 0].plot(wn, color=COLORS['blue'], linewidth=0.5)
axes[0, 0].axhline(0, color=COLORS['gray'], linewidth=0.5, linestyle='--')
axes[0, 0].set_title('White noise (WN)', fontsize=9, fontweight='bold')
axes[0, 0].set_ylabel(r'$\varepsilon_t$')

acf_wn = acf(wn, nlags=20)
axes[0, 1].bar(range(len(acf_wn)), acf_wn, color=COLORS['blue'], width=0.5, alpha=0.7)
ci = 1.96 / np.sqrt(n)
axes[0, 1].axhline(ci, color=COLORS['red'], linewidth=0.8, linestyle='--')
axes[0, 1].axhline(-ci, color=COLORS['red'], linewidth=0.8, linestyle='--')
axes[0, 1].set_title('ACF: White noise', fontsize=9, fontweight='bold')
axes[0, 1].set_ylabel(r'$\hat{\rho}(h)$')

axes[1, 0].plot(rw, color=COLORS['red'], linewidth=0.8)
axes[1, 0].set_title('Random walk (RW)', fontsize=9, fontweight='bold')
axes[1, 0].set_xlabel('Time')
axes[1, 0].set_ylabel(r'$X_t$')

acf_rw = acf(rw, nlags=20)
axes[1, 1].bar(range(len(acf_rw)), acf_rw, color=COLORS['red'], width=0.5, alpha=0.7)
axes[1, 1].set_title('ACF: Random walk (slow decay)', fontsize=9, fontweight='bold')
axes[1, 1].set_xlabel('Lag')
axes[1, 1].set_ylabel(r'$\hat{\rho}(h)$')

fig.tight_layout(rect=[0, 0.05, 1, 1])
save_chart(fig, 'ch1_wn_rw')
plt.show()

In [ ]:
# Chart: rw_vs_stationary
# ACF comparison: stationary vs random walk
np.random.seed(42)
n = 300
stationary = generate_ar1(n, phi=0.7, sigma=1.0)
rw = generate_random_walk(n)
nlags = 25

fig, axes = plt.subplots(2, 2, figsize=(8, 4.0))

axes[0, 0].plot(stationary, color=COLORS['blue'], linewidth=0.7)
axes[0, 0].axhline(0, color=COLORS['gray'], linewidth=0.5, linestyle='--')
axes[0, 0].set_title('Stationary AR(1) series', fontsize=9, fontweight='bold')
axes[0, 0].set_ylabel(r'$X_t$')

acf_stat = acf(stationary, nlags=nlags)
axes[0, 1].bar(range(len(acf_stat)), acf_stat, color=COLORS['blue'], width=0.5, alpha=0.7)
ci = 1.96 / np.sqrt(n)
axes[0, 1].axhline(ci, color=COLORS['red'], linewidth=0.8, linestyle='--')
axes[0, 1].axhline(-ci, color=COLORS['red'], linewidth=0.8, linestyle='--')
axes[0, 1].set_title('ACF: Fast decay', fontsize=9, fontweight='bold')
axes[0, 1].set_ylabel(r'$\hat{\rho}(h)$')

axes[1, 0].plot(rw, color=COLORS['red'], linewidth=0.8)
axes[1, 0].set_title('Random walk', fontsize=9, fontweight='bold')
axes[1, 0].set_xlabel('Time')
axes[1, 0].set_ylabel(r'$X_t$')

acf_rw = acf(rw, nlags=nlags)
axes[1, 1].bar(range(len(acf_rw)), acf_rw, color=COLORS['red'], width=0.5, alpha=0.7)
axes[1, 1].set_title('ACF: Very slow decay', fontsize=9, fontweight='bold')
axes[1, 1].set_xlabel('Lag')
axes[1, 1].set_ylabel(r'$\hat{\rho}(h)$')

fig.tight_layout(rect=[0, 0.05, 1, 1])
save_chart(fig, 'rw_vs_stationary')
plt.show()

In [ ]:
# Chart: ch1_trend_types
# Deterministic vs stochastic trends
np.random.seed(42)
n = 200
t = np.arange(n)

fig, axes = plt.subplots(1, 3, figsize=(8, 2.5))

y_lin = 10 + 0.3*t + np.random.normal(0, 3, n)
axes[0].plot(y_lin, color=COLORS['blue'], linewidth=0.6)
axes[0].plot(10 + 0.3*t, color=COLORS['red'], linewidth=1.2, linestyle='--')
axes[0].set_title('Linear trend', fontsize=9, fontweight='bold')
axes[0].set_xlabel('Time')
axes[0].set_ylabel(r'$X_t$')

y_exp = 10 * np.exp(0.01*t) + np.random.normal(0, 3, n)
axes[1].plot(y_exp, color=COLORS['blue'], linewidth=0.6)
axes[1].plot(10 * np.exp(0.01*t), color=COLORS['red'], linewidth=1.2, linestyle='--')
axes[1].set_title('Exponential trend', fontsize=9, fontweight='bold')
axes[1].set_xlabel('Time')

y_stoch = generate_random_walk(n, drift=0.1)
axes[2].plot(y_stoch, color=COLORS['blue'], linewidth=0.8)
axes[2].plot(0.1*t, color=COLORS['red'], linewidth=1.2, linestyle='--')
axes[2].set_title('Stochastic trend (RW)', fontsize=9, fontweight='bold')
axes[2].set_xlabel('Time')

fig.tight_layout(rect=[0, 0.05, 1, 1])
save_chart(fig, 'ch1_trend_types')
plt.show()

In [ ]:
# Chart: ch1_unit_root_series
# Unit root series variants
np.random.seed(42)
n = 200

fig, axes = plt.subplots(1, 3, figsize=(8, 2.5))

rw = generate_random_walk(n)
axes[0].plot(rw, color=COLORS['blue'], linewidth=0.8)
axes[0].set_title(r'RW: $X_t = X_{t-1} + \varepsilon_t$', fontsize=9, fontweight='bold')
axes[0].set_xlabel('Time')
axes[0].set_ylabel(r'$X_t$')

rw_d = generate_random_walk(n, drift=0.1)
axes[1].plot(rw_d, color=COLORS['red'], linewidth=0.8)
axes[1].set_title(r'RW + drift: $X_t = c + X_{t-1} + \varepsilon_t$', fontsize=9, fontweight='bold')
axes[1].set_xlabel('Time')

y_near = np.zeros(n)
eps = np.random.normal(0, 1, n)
for t_idx in range(1, n):
    y_near[t_idx] = 0.98 * y_near[t_idx-1] + eps[t_idx]
axes[2].plot(y_near, color=COLORS['green'], linewidth=0.8)
axes[2].set_title(r'Near unit root: $\phi = 0.98$', fontsize=9, fontweight='bold')
axes[2].set_xlabel('Time')

fig.tight_layout(rect=[0, 0.05, 1, 1])
save_chart(fig, 'ch1_unit_root_series')
plt.show()